# 实验对比：自定义框架 vs. 默认 LC2013

[cite_start]本 Notebook 用于运行 A/B 对比测试，评估论文中提出的自定义换道框架 [cite: 257-376][cite_start]（实验 B）与 SUMO 的默认 LC2013 [cite: 161-173] 换道模型（实验 A）之间的性能差异。

**对比指标:**
1.  **吞吐量 (Throughput)**
2.  **平均速度 (Average Speed)**
3.  [cite_start]**2D-TTC 安全性 (2D-TTC)** [cite: 307-313]

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# 【修改1】使用标准 tqdm，兼容终端和 Notebook
from tqdm import tqdm 
import traci
import traci.constants as tc

# =================================================================
# 1. 基础配置与导入
# =================================================================
CURRENT_DIR = os.path.abspath(os.getcwd())
PROJECT_ROOT = os.path.dirname(CURRENT_DIR) if "controller" not in os.listdir(CURRENT_DIR) else CURRENT_DIR

if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)

from controller.config import CONFIG
from controller.controllers.lane_manager import LaneManager
from controller.controllers.vehicle_controller import VehicleController, MODE_NO_CONTROL, MODE_BASELINE, MODE_CUSTOM
from controller.environment.twod_ttc_calculator import calculate_2d_ttc

# =================================================================
# 4. 全局配置
# =================================================================
SCENARIO_PATH = os.path.join(PROJECT_ROOT, "scenario")
SUMO_CONFIG_FILE = os.path.join(SCENARIO_PATH, "test.sumocfg")

CONTROL_EDGES = ["2", "3", "4", "5", "6", "7", "8", "9", "10", "11"]
CONTROL_EDGES_SET = set(CONTROL_EDGES) 

CONTROLLED_LANE_INDEX = 0
MIDDLE_LANE_INDEX = 1
FIXED_POLICY_M = 5  
FIXED_POLICY_N = 5  
CRITICAL_TTC_THRESHOLD = 5.0 
MAX_SIMULATION_STEPS = 18000
DATA_RECORD_FREQUENCY = 3000 

# =================================================================
# 5. 高性能 TTC 计算器
# =================================================================
class FastTTCCalculator:
    def __init__(self, vehicle_length=5.0, vehicle_width=1.8):
        self.L = vehicle_length
        self.W = vehicle_width

    def calculate_global_metrics(self):
        # 1. 获取全量订阅数据
        raw_results = traci.vehicle.getAllSubscriptionResults()
        
        states = {}
        target_veh_ids = []

        # --- 阶段一：数据解析与预处理 ---
        for vid, data in raw_results.items():
            if not data: continue
            
            if tc.VAR_ROAD_ID not in data or tc.VAR_POSITION not in data:
                continue

            edge_id = data[tc.VAR_ROAD_ID]
            if edge_id not in CONTROL_EDGES_SET:
                continue

            pos = data[tc.VAR_POSITION]
            speed = data[tc.VAR_SPEED]
            angle_sumo = data[tc.VAR_ANGLE]
            angle_rad = np.radians((90.0 - angle_sumo) % 360.0)
            
            states[vid] = (pos[0], pos[1], speed, angle_rad, self.L, self.W)
            target_veh_ids.append(vid)

        if len(target_veh_ids) < 2:
            return np.nan, 0

        ttc_values = []
        critical_count = 0

        # --- 阶段二：配对与 TTC 计算 ---
        for vid in target_veh_ids:
            # 直接从订阅数据中获取 Leader 信息 (依赖于 run_simulation 中的带参订阅)
            # 默认返回 400 (无穷远)
            raw_leader_data = raw_results[vid].get(tc.VAR_LEADER, None)
            
            current_safety_score = 1.0
            
            if raw_leader_data and isinstance(raw_leader_data, (list, tuple)):
                leader_id, dist = raw_leader_data              
                
                if leader_id in states:
                    state_i = states[vid]       # Ego
                    state_j = states[leader_id] # Leader
                    
                    v_i = state_i[2]
                    v_j = state_j[2]
                    
                    # 预筛选
                    if v_i <= v_j and dist > 10.0:
                        ttc = float('inf')
                    else:
                        try:
                            ttc = calculate_2d_ttc(state_i, state_j)
                        except Exception:
                            dv = max(v_i - v_j, 0.001)
                            ttc = dist / dv

                    if 0 < ttc < CRITICAL_TTC_THRESHOLD:
                        current_safety_score = ttc / CRITICAL_TTC_THRESHOLD
                        critical_count += 1
                    else:
                        current_safety_score = 1.0
            
            ttc_values.append(current_safety_score)

        avg_ttc = np.mean(ttc_values) if ttc_values else np.nan
        return avg_ttc, critical_count

# =================================================================
# 6. 仿真主函数 (集成进度条)
# =================================================================
def run_simulation(control_mode: int, label: str):
    # 移除原本的 print，改用进度条描述
    # print(f"🚀 [Start] {label}...") 
    
    try: traci.close()
    except: pass

    sumo_cmd = ["sumo", "-c", SUMO_CONFIG_FILE, "--step-length", "0.1", "--no-warnings", "true", "--no-step-log", "true", "--seed", "9497"]
    if sys.platform.startswith("win"): sumo_cmd[0] += ".exe"
    
    traci.start(sumo_cmd, label="default")

    lane_manager = LaneManager(CONTROL_EDGES, CONTROLLED_LANE_INDEX, MIDDLE_LANE_INDEX)
    veh_controller = VehicleController(CONTROL_EDGES)
    ttc_calc = FastTTCCalculator()

    if control_mode == MODE_NO_CONTROL:
        print(">>> [Mode 0] 初始化: 重置为混行模式，清除所有专用道限制。")
        lane_manager.reset_to_mixed_traffic()

    elif control_mode == MODE_BASELINE:
        print(f">>> [Mode 1] 初始化: 应用固定策略 (m={FIXED_POLICY_M}, n={FIXED_POLICY_N})")
        lane_manager.apply_lane_strategy(FIXED_POLICY_M, FIXED_POLICY_N)
        
    else: 
        # MODE_BASELINE (1) 或 MODE_CUSTOM (2)
        print(f">>> [Mode {control_mode}] 初始化: 应用固定策略 (m={FIXED_POLICY_M}, n={FIXED_POLICY_N})")
        lane_manager.apply_lane_strategy(FIXED_POLICY_M, FIXED_POLICY_N)

    results = []
    step = 0
    sum_throughput = 0
    
    try:
 
        with tqdm(total=MAX_SIMULATION_STEPS, desc=f"🚀 {label:<15}", unit="step") as pbar:     
            while traci.simulation.getMinExpectedNumber() > 0 and step <= MAX_SIMULATION_STEPS:             
                traci.simulationStep()
                
                # --- 自动订阅 ---
                departed = traci.simulation.getDepartedIDList()
                if departed:
                    for vid in departed:
                        traci.vehicle.subscribe(vid, [
                            tc.VAR_POSITION, 
                            tc.VAR_SPEED, 
                            tc.VAR_ANGLE,
                            tc.VAR_ROAD_ID,
                            tc.VAR_LEADER 
                        ], parameters={tc.VAR_LEADER: 400.0})

                if control_mode != MODE_NO_CONTROL:
                # Mode 1 & 2: 必须运行，因为需要清理 HCL 上的滞留车辆
                    lane_manager.step()

                # --- 控制逻辑 ---               
                if control_mode == MODE_CUSTOM:
                    veh_controller.update_vehicle_states(
                        lane_manager.get_active_hml_lanes(),
                        lane_manager.get_active_cdl_lanes(),
                        lane_manager.get_active_middle_lanes(),
                        lane_manager.get_cdl_start_edge(),
                        control_mode=MODE_CUSTOM
                    )
                elif control_mode == MODE_NO_CONTROL:
                    veh_controller.enforce_nocontrol_mode()
                elif control_mode == MODE_BASELINE:
                    veh_controller.enforce_baseline_mode()

                # --- 数据记录 ---
                sum_throughput += traci.simulation.getArrivedNumber()

                if step % DATA_RECORD_FREQUENCY == 0:
                    subs = traci.vehicle.getAllSubscriptionResults()
                    speeds = [d[tc.VAR_SPEED] for d in subs.values() if tc.VAR_SPEED in d]
                    mean_speed = np.mean(speeds) if speeds else np.nan
                    
                    avg_ttc, crit_count = ttc_calc.calculate_global_metrics()
                    
                    results.append({
                        "step": step,
                        "time": traci.simulation.getTime(),
                        "mode": label,
                        "throughput": sum_throughput,
                        "mean_speed": mean_speed,
                        "avg_ttc": avg_ttc,
                        "crit_events": crit_count
                    })

                    # 【修改3】实时更新进度条尾部信息 (非常实用！)
                    # 显示：累计流量、当前平均车速、当前安全指数
                    pbar.set_postfix({
                        "Thr": f"{int(sum_throughput)}",
                        "Spd": f"{mean_speed:.1f}m/s",
                        "Safe": f"{avg_ttc:.2f}"
                    })
                
                step += 1
                # 【修改4】更新进度条
                pbar.update(1)
            
    except Exception as e:
        print(f"  [错误] 仿真中断: {e}")
        import traceback
        traceback.print_exc()
    finally:
        try: traci.close()
        except: pass
        
    # print(f"  [结束] {label} 完成...") # 进度条完成后无需额外打印，或保留均可
    return pd.DataFrame(results)

# =================================================================
# 7. 主执行入口
# =================================================================
if __name__ == "__main__":
    
    if not os.path.exists(SUMO_CONFIG_FILE):
        print(f"错误: 找不到配置文件 {SUMO_CONFIG_FILE}")
    else:
        df_no_control = run_simulation(MODE_NO_CONTROL, "No_Control")
        df_baseline = run_simulation(MODE_BASELINE, "Baseline")
        df_custom = run_simulation(MODE_CUSTOM, "Custom")
        
        # ... (后续处理逻辑保持不变)
        dfs = [df_no_control, df_baseline, df_custom]
        if any(not df.empty for df in dfs):
            df_no_control.to_csv("result_no_control.csv", index=False)
            df_baseline.to_csv("result_baseline.csv", index=False)
            df_custom.to_csv("result_custom.csv", index=False)
            
            print("\n" + "="*80)
            print("SIMULATION RESULTS SUMMARY")
            print("="*80)
            print(f"{'Mode':<18} | {'Steps':<8} | {'Total Thr':<12} | {'Avg Spd':<10} | {'Avg TTC':<10}| {'Crit_Ev':<12}")
            print("-" * 80)
            
            metrics = {"No Control": df_no_control, "Baseline": df_baseline, "Custom": df_custom}
            for name, df in metrics.items():
                if df.empty:
                    print(f"{name:<18} | {'FAILED':<40}")
                    continue
                total_thr = df['throughput'].values[-1] if len(df) > 0 else 0
                avg_spd = df['mean_speed'].mean()
                avg_ttc = df['avg_ttc'].mean()
                crit_ev = df['crit_events'].values[-1] if len(df) > 0 else 0
                steps_run = df['step'].iloc[-1] if len(df) > 0 else 0
                print(f"{name:<18} | {steps_run:<8} | {total_thr:<12.0f} | {avg_spd:<10.2f} | {avg_ttc:<10.6f} | {crit_ev:<12.0f}")
            print("-" * 80)
        else:
            print("所有仿真均未产生有效数据。")

LaneManager (V-Paper) 初始化完毕，管理 10 个 SCL 路段。
VehicleController: 正在缓存 vType 参数...
>>> [Mode 0] 初始化: 重置为混行模式，清除所有专用道限制。


🚀 No_Control     : 18001step [15:25, 19.46step/s, Thr=1316, Spd=4.4m/s, Safe=1.00]                         


LaneManager (V-Paper) 初始化完毕，管理 10 个 SCL 路段。
VehicleController: 正在缓存 vType 参数...
>>> [Mode 1] 初始化: 应用固定策略 (m=5, n=5)


🚀 Baseline       : 18001step [07:20, 40.88step/s, Thr=1809, Spd=10.5m/s, Safe=1.00]                         


LaneManager (V-Paper) 初始化完毕，管理 10 个 SCL 路段。
VehicleController: 正在缓存 vType 参数...
>>> [Mode 2] 初始化: 应用固定策略 (m=5, n=5)


🚀 Custom         : 18001step [03:50, 78.04step/s, Thr=1767, Spd=14.8m/s, Safe=1.00]                         



SIMULATION RESULTS SUMMARY
Mode               | Steps    | Total Thr    | Avg Spd    | Avg TTC   
--------------------------------------------------------------------------------
No Control         | 18000    | 1316         | 11.23      | 0.999989   | 0           
Baseline           | 18000    | 1809         | 16.28      | 0.999905   | 2           
Custom             | 18000    | 1767         | 17.88      | 0.990899   | 1           
--------------------------------------------------------------------------------


# 结果分析与可视化

In [ ]:
def summarize_results(df: pd.DataFrame, name: str):
    if df.empty:
        print(f"--- {name} 结果 (无数据) ---")
        return
        
    print(f"\n--- {name} 结果汇总 ---")
    
    # 1. 吞吐量
    total_throughput = df['throughput_veh'].sum()
    print(f"  总吞吐量 (辆): {total_throughput}")
    
    # 2. 平均速度 (m/s -> km/h)
    avg_speed_kph = (df['mean_speed_mps'].mean() * 3.6) # .mean() 自动忽略 nans
    print(f"  平均速度 (km/h): {avg_speed_kph:.2f}")
    
    # 3. 2D-TTC
    total_critical_events = df['critical_ttc_events'].sum()
    print(f"  危险 2D-TTC 事件总数 (TTC < {CRITICAL_TTC_THRESHOLD}s): {total_critical_events}")

# 打印两个实验的汇总
summarize_results(df_baseline, "A: 默认 LC2013 ")
summarize_results(df_custom, "B: 自定义框架 ")

In [ ]:
if not df_baseline.empty and not df_custom.empty:
    plt.figure(figsize=(12, 6))
    
    # 计算累计吞吐量
    df_baseline['cumulative_throughput'] = df_baseline['throughput_veh'].cumsum()
    df_custom['cumulative_throughput'] = df_custom['throughput_veh'].cumsum()
    
    plt.plot(df_baseline['time_s'], df_baseline['cumulative_throughput'], label='默认 LC2013 ')
    plt.plot(df_custom['time_s'], df_custom['cumulative_throughput'], label='自定义框架 ', linestyle='--')
    
    plt.title('对比 1: 累计吞吐量', fontsize=16)
    plt.xlabel('仿真时间 (秒)')
    plt.ylabel('累计吞吐量 (辆)')
    plt.legend()
    plt.grid(True)
    plt.show()
else:
    print("数据不完整，跳过绘制吞吐量图表。")

In [ ]:
if not df_baseline.empty and not df_custom.empty:
    plt.figure(figsize=(12, 6))
    
    # 使用滚动平均 (rolling mean) 使曲线平滑
    ROLLING_WINDOW = 300 # 30秒 (300 步 * 0.1s/步)
    
    baseline_speed_kph = (df_baseline['mean_speed_mps'] * 3.6).rolling(window=ROLLING_WINDOW).mean()
    custom_speed_kph = (df_custom['mean_speed_mps'] * 3.6).rolling(window=ROLLING_WINDOW).mean()
    
    plt.plot(df_baseline['time_s'], baseline_speed_kph, label=f'默认 LC2013  ({ROLLING_WINDOW}步滚动平均)')
    plt.plot(df_custom['time_s'], custom_speed_kph, label=f'自定义框架  ({ROLLING_WINDOW}步滚动平均)', linestyle='--')
    
    plt.title('对比 2: 平均速度', fontsize=16)
    plt.xlabel('仿真时间 (秒)')
    plt.ylabel('平均速度 (km/h)')
    plt.legend()
    plt.grid(True)
    plt.ylim(bottom=0) # 速度不能为负
    plt.show()
else:
    print("数据不完整，跳过绘制速度图表。")

In [ ]:
if not df_baseline.empty and not df_custom.empty:
    plt.figure(figsize=(10, 6))
    
    # 比较两个实验中 *发生* 危险TTC事件的 *总数*
    # (这比平均TTC值更能反映安全性)
    events_baseline = df_baseline['critical_ttc_events'].sum()
    events_custom = df_custom['critical_ttc_events'].sum()
    
    labels = ['默认 LC2013 ', '自定义框架 ']
    values = [events_baseline, events_custom]
    
    plt.bar(labels, values, color=['blue', 'orange'])
    
    plt.title(f'对比 3: 危险 2D-TTC 事件总数 (TTC < {CRITICAL_TTC_THRESHOLD}s)', fontsize=16)
    plt.ylabel('事件总数 (越低越安全)')
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    
    for i, v in enumerate(values):
        plt.text(i, v + (max(values) * 0.01), str(v), ha='center', fontweight='bold')
        
    plt.show()
else:
    print("数据不完整，跳过绘制 2D-TTC 图表。")